In [1]:
import pandas as pd

## Assume taxi:mode1, FHV: mode2, shared FHV: mode3, PT: mode4, walking: mode5

In [21]:
fhv = pd.read_csv('final_fhv.csv', index_col=0)
print(fhv.shape)
fhv = fhv.groupby(['PUlocationID', 'DOlocationID']).mean().reset_index()
print(fhv.shape)
fhv.head()

(61109, 5)
(60615, 5)


,PUlocationID,DOlocationID,ride_counts,trip_duation_mean (seconds),price
0,1,4,17,2578.058824,74.5
1,1,5,3,3337.333333,55.5
2,1,6,7,4181.428571,57.5
3,1,7,3,4672.000000,104.0
4,1,10,1,3052.000000,122.0


In [22]:
sharedfhv = pd.read_csv('final_shared_fhv.csv',index_col=0)
print(sharedfhv.shape)
sharedfhv = sharedfhv.groupby(['PUlocationID', 'DOlocationID']).mean().reset_index()
print(sharedfhv.shape)
sharedfhv.head()

(53598, 5)
(53138, 5)


,PUlocationID,DOlocationID,ride_counts,trip_duation_mean (seconds),price
0,1,4,10,3791.000000,68.5
1,1,13,4,1998.250000,63.5
2,1,24,11,3769.545455,80.5
3,1,25,2,3497.500000,69.5
4,1,33,3,3169.000000,68.5


In [23]:
taxi = pd.read_csv('final_taxi.csv')
print(taxi.shape)
taxi = taxi.groupby(['PULocationID', 'DOLocationID']).mean().reset_index()
print(taxi.shape)
taxi.head()

(38775, 6)
(38775, 6)


,PULocationID,DOLocationID,ride_counts_sum,ODtrip_dur_mean,ODfare_amount_mean,ODmean_passengers_per_trip
0,1,1,2657,210.982311,75.286786,1.481746
1,1,13,3,3161.000000,39.503333,1.666667
2,1,23,1,1364.000000,51.000000,1.000000
3,1,24,1,2772.000000,59.000000,1.000000
4,1,25,1,5908.000000,75.000000,1.000000


In [25]:
PT = pd.read_csv('final_PublicTransit.csv').drop(['Unnamed: 0', 'lon_pickup', 'lat_pickup', 'lon_dropoff', 'lat_dropoff'], axis=1)
#PT = PT[['TZ_pickup', 'TZ_dropoff', 'trip_duration(s)', 'price']]
print(PT.shape)
PT = PT.groupby(['TZ_pickup', 'TZ_dropoff']).mean().reset_index()
PT = PT[['TZ_pickup', 'TZ_dropoff', 'trip_duration(s)', 'price']]
print(PT.shape)
PT.head()

(65536, 15)
(65027, 4)


,TZ_pickup,TZ_dropoff,trip_duration(s),price
0,3,1,7680,5.50
1,3,2,8400,8.25
2,3,4,4980,5.50
3,3,5,9840,8.25
4,3,6,6600,5.50


In [26]:
walking = pd.read_csv('final_walking.csv').drop(['Unnamed: 0'], axis=1)
print(walking.shape)
walking = walking.groupby(['LocationID_pickup', 'LocationID_dropoff']).mean().reset_index()
print(walking.shape)
walking.head()

(67377, 4)
(65839, 4)


,LocationID_pickup,LocationID_dropoff,time,distance
0,1,2,31379.0,65155.0
1,1,3,34145.0,71302.0
2,1,4,25529.0,49169.0
3,1,5,42701.0,50553.0
4,1,6,31821.0,39772.0


In [27]:
wage = pd.read_csv('taxi_zone_wages_raw.csv', index_col=0).set_index('taxi_zone')
wage = wage.drop('total_count', axis=1)
print(wage.shape)
wage = wage.loc[(wage!=0).any(1)].reset_index()
print(wage.shape)
# print(wage.shape)
# wage = wage.groupby(['OBJECTID']).mean().reset_index()
# print(wage.shape)
wage.head()

(261, 11)
(258, 12)


,taxi_zone,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000
0,3.0,791,1136,2045,890,1308,1867,2117,2960,1851,2336,1621
1,4.0,1403,1677,2318,1300,1277,1649,2068,2787,1360,2391,1751
2,5.0,200,161,303,216,219,517,1016,1563,963,1809,2081
3,6.0,1293,836,1183,1051,982,1753,2252,3066,3036,3874,6297
4,7.0,5560,6148,7198,6415,5057,9139,12866,18294,15610,22284,42329


_____________________________________________________________________

In [29]:
def data_with_wage(modeData, usedCol, wage, leftOn, rightOn):
    useData = modeData[usedCol]
    print('The origin shape of data is', useData.shape)
#     print('The origin length of wage data is', len(wage))
#     wage = wage.dropna()
#     print('The length of wage data after dropna is', len(wage))
    useData[leftOn] = useData[leftOn].astype(int)
    wage[rightOn] = wage[rightOn].astype(int)
    combine = pd.merge(useData, wage, left_on = leftOn, right_on=rightOn, how='left')
    combine = combine.drop(['taxi_zone'], axis=1)
    combine['ODpair'] = combine[usedCol[0]].astype(str) + '-' + combine[usedCol[1]].astype(str)
    combine = combine.dropna()
    return combine
#     pt['price'] = pt['transfer_list']*2.75 + 2.75
#     locationRI = location.reset_index(drop=True)
#     combine1 = pd.merge(locationRI, pt, left_index =True, right_index=True)
#     combine2 = pd.merge(combine1, wage, left_on='LocationID_pickup', right_on='OBJECTID', how='left')
#     combine2 = combine2.drop(['Unnamed: 0', 'transfer_list', 'modelist_list', 'OBJECTID'], axis=1)
#     combine2.dropna(inplace=True)
#     combine2['duration_list'] = combine2['duration_list'].map(lambda x: int(x.split('T')[1].split('H')[0])*3600 
#                                                               + int(x.split('H')[1].split('M')[0])*60 if len(x)>5 
#                                                               else int(x.split('T')[1].split('M')[0])*60)

#     combine2.rename(columns={'LocationID_pickup':'TZ_pickup', 'LocationID_dropoff':'TZ_dropoff', 'duration_list':'trip_duration(s)'}, inplace=True)
#     return combine2

In [30]:
final_fhv = data_with_wage(fhv, ['PUlocationID', 'DOlocationID', 'trip_duation_mean (seconds)', 'price'], wage, 'PUlocationID', 'taxi_zone').rename(columns = {'trip_duation_mean (seconds)':'duration'})
print(final_fhv.shape)
print(len(final_fhv['PUlocationID'].unique()))
print(len(final_fhv['DOlocationID'].unique()))
final_fhv['mode'] = 2
final_fhv['nest'] = 1
final_fhv.head()

The origin shape of data is (60615, 4)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


(60135, 16)
256
259


,PUlocationID,DOlocationID,duration,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,ODpair,mode,nest
223,3,1,3355.372881,126.0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-1,2,1
224,3,4,2381.700000,64.0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-4,2,1
225,3,6,2806.750000,126.0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-6,2,1
226,3,7,1831.304348,47.0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-7,2,1
227,3,9,1172.827586,39.0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-9,2,1


In [31]:
final_sharedfhv =  data_with_wage(sharedfhv, ['PUlocationID', 'DOlocationID', 'trip_duation_mean (seconds)', 'price'], wage, 'PUlocationID', 'taxi_zone').rename(columns = {'trip_duation_mean (seconds)':'duration'})
print(final_sharedfhv.shape)
print(len(final_sharedfhv['PUlocationID'].unique()))
print(len(final_sharedfhv['DOlocationID'].unique()))
final_sharedfhv['mode'] = 3
final_sharedfhv['nest'] = 1
final_sharedfhv.head()

The origin shape of data is (53138, 4)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


(52807, 16)
256
258


,PUlocationID,DOlocationID,duration,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,ODpair,mode,nest
78,3,1,3745.200000,120.0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-1,3,1
79,3,4,2713.000000,61.5,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-4,3,1
80,3,7,2229.540541,45.0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-7,3,1
81,3,9,1878.000000,38.0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-9,3,1
82,3,10,4173.000000,66.5,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-10,3,1


In [32]:
final_taxi =  data_with_wage(taxi, ['PULocationID', 'DOLocationID', 'ODtrip_dur_mean', 'ODfare_amount_mean']
                             , wage, 'PULocationID', 'taxi_zone').rename(columns = {'PULocationID':'PUlocationID', 
                                                                                   'DOLocationID': 'DOlocationID',
                                                                                   'ODtrip_dur_mean':'duration',
                                                                                  'ODfare_amount_mean': 'price'})
print(final_taxi.shape)
print(len(final_taxi['PUlocationID'].unique()))
print(len(final_taxi['DOlocationID'].unique()))
final_taxi['mode'] = 1
final_taxi['nest'] = 1
final_taxi.head()

The origin shape of data is (38775, 4)
(37978, 16)
256
262


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,PUlocationID,DOlocationID,duration,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,ODpair,mode,nest
75,3,1,3399.000000,100.000000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-1,1,1
76,3,3,390.536585,16.647073,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-3,1,1
77,3,7,2872.857143,43.157143,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-7,1,1
78,3,18,914.400000,12.900000,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-18,1,1
79,3,20,548.666667,10.166667,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-20,1,1


In [35]:
final_PT =  data_with_wage(PT, ['TZ_pickup', 'TZ_dropoff', 'trip_duration(s)', 'price']
                             , wage, 'TZ_pickup', 'taxi_zone').rename(columns = {'TZ_pickup':'PUlocationID', 
                                                                                   'TZ_dropoff': 'DOlocationID',
                                                                                   'trip_duration(s)':'duration',
                                                                                  })
print(final_PT.shape)
print(len(final_PT['PUlocationID'].unique()))
print(len(final_PT['DOlocationID'].unique()))
final_PT['mode'] = 4
final_PT['nest'] = 2
final_PT.head()

The origin shape of data is (65027, 4)
(65027, 16)
255
259


,PUlocationID,DOlocationID,duration,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,ODpair,mode,nest
0,3,1,7680,5.50,791,1136,2045,890,1308,1867,2117,2960,1851,2336,1621,3-1,4,2
1,3,2,8400,8.25,791,1136,2045,890,1308,1867,2117,2960,1851,2336,1621,3-2,4,2
2,3,4,4980,5.50,791,1136,2045,890,1308,1867,2117,2960,1851,2336,1621,3-4,4,2
3,3,5,9840,8.25,791,1136,2045,890,1308,1867,2117,2960,1851,2336,1621,3-5,4,2
4,3,6,6600,5.50,791,1136,2045,890,1308,1867,2117,2960,1851,2336,1621,3-6,4,2


In [75]:
# final_PT = PT.rename(columns={'TZ_pickup':'PUlocationID', 'TZ_dropoff':'DOlocationID', 'trip_duration(s)':'duration'})
# print(final_PT.shape)
# print(len(final_PT['PUlocationID'].unique()))
# print(len(final_PT['DOlocationID'].unique()))
# final_PT['mode'] = 4
# final_PT['ODpair'] = final_PT['PUlocationID'].astype(str) + '-' +  final_PT['DOlocationID'].astype(str)
# final_PT['nest'] = 2
# final_PT.head()

In [36]:
walking['price'] = 0
final_walking =  data_with_wage(walking, ['LocationID_pickup', 'LocationID_dropoff', 'time', 'price'], wage, 
                                'LocationID_pickup', 'taxi_zone').rename(columns = {'LocationID_pickup':'PUlocationID', 
                                                                                   'LocationID_dropoff': 'DOlocationID', 
                                                                                   'time':'duration'})
print(final_walking.shape)
print(len(final_walking['PUlocationID'].unique()))
print(len(final_walking['DOlocationID'].unique()))
final_walking['mode'] = 5
# final_walking['ODpair'] = final_PT['PUlocationID'].astype(str) + '-' +  final_PT['DOlocationID'].astype(str)
final_walking['nest'] = 3
final_walking.head()


The origin shape of data is (65839, 4)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


(65323, 16)
254
260


,PUlocationID,DOlocationID,duration,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,ODpair,mode,nest
516,3,1,34145.0,0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-1,5,3
517,3,2,21228.0,0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-2,5,3
518,3,4,13556.0,0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-4,5,3
519,3,5,32330.0,0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-5,5,3
520,3,6,21450.0,0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-6,5,3


In [37]:
dataAll = [final_fhv, final_sharedfhv, final_taxi, final_PT, final_walking]
# dataAll['PUlocationID'] = dataAll['PUlocationID'].astype(int)
# dataAll['PUlocationID'] = dataAll['PUlocationID'].astype(int)
final_allMode = pd.concat(dataAll).sort_values(['PUlocationID', 'DOlocationID', 'ODpair', 'mode']).reset_index(drop=True)
final_allMode.drop_duplicates(keep = False, inplace = True)
print(final_allMode.shape)
final_allMode.head()

(281270, 18)


,PUlocationID,DOlocationID,duration,price,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,ODpair,mode,nest
0,3,1,3399.000000,100.0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-1,1,1
1,3,1,3355.372881,126.0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-1,2,1
2,3,1,3745.200000,120.0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-1,3,1
3,3,1,7680.000000,5.5,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-1,4,2
4,3,1,34145.000000,0.0,791.0,1136.0,2045.0,890.0,1308.0,1867.0,2117.0,2960.0,1851.0,2336.0,1621.0,3-1,5,3


In [38]:
final_fhv.to_csv('final_fhv_with_wage.csv')
final_sharedfhv.to_csv('final_sharedfhv_with_wage.csv')
final_taxi.to_csv('final_taxi_with_wage.csv')
final_PT.to_csv('final_PT_with_wage.csv')
final_walking.to_csv('final_walking_with_wage.csv')

In [39]:
final_allMode.to_csv('final_allMode_with_wage.csv')

#### Groud truth proprotion:

In [44]:
wage = pd.read_csv('taxi_zone_wages_raw.csv', index_col=0).set_index('taxi_zone').reset_index()
amount = wage[['taxi_zone', 'total_count']]
amount['taxi_zone'] = amount['taxi_zone'].astype(int)
amount.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,taxi_zone,total_count
0,2,0
1,3,18922
2,4,19981
3,5,9048
4,6,25623


In [45]:
acs = pd.read_csv('ACS_transportation_choice.csv')
acs = acs.rename(columns={'FHV proportion':'P(mode1)',
                         'FHV_shared proportion':'P(mode2)',
                         'Taxi proportion':'P(mode3)',
                         'pt':'P(mode4)',
                         'walked':'P(mode5)'})
# acs = acs.set_index('taxi_zone')
acs.drop('Unnamed: 0', axis=1, inplace=True)
print(acs.shape)
acs.head()

(241, 6)


,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
0,3,0.009394,0.003862,0.000061,0.880923,0.105760
1,4,0.010074,0.006050,0.003064,0.666539,0.314272
2,5,0.005482,0.000883,0.000034,0.977816,0.015785
3,6,0.006416,0.001578,0.000092,0.894894,0.097021
4,7,0.004083,0.001076,0.000731,0.914596,0.079514


In [48]:
acs_pop = pd.merge(amount, acs, left_on='taxi_zone', right_on='taxi_zone')
for i in acs_pop.columns[2:]:
    acs_pop[i] = acs_pop[i]*acs_pop['total_count']
acs_pop = acs_pop.drop('total_count', axis=1)
acs_pop.head()

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
0,3,177.762595,73.068204,1.150001,16668.822855,2001.196346
1,4,201.297419,120.885755,61.223127,13318.123341,6279.470358
2,5,49.601278,7.992997,0.306749,8847.276451,142.822526
3,6,164.390442,40.427858,2.346380,22929.859149,2485.976170
4,7,616.150103,162.371679,110.267353,138012.557541,11998.653324


In [50]:
acs.dtypes

taxi_zone      int64
P(mode1)     float64
P(mode2)     float64
P(mode3)     float64
P(mode4)     float64
P(mode5)     float64
dtype: object

In [51]:
acs_pop.to_csv('final_acs_transportation_choice.csv')